# WRI Global Power Plant Database Ingestion Pipeline: Step 2 - Data Loading
Copyright (C) 2021 OS-Climate

This notebook covers the second step of the pipeline:
* Load versioned source data from Pachyderm
* Get data into a dataframe 
* Create a Trino client and load the data

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

In [1]:
%%capture pipoutput

# For loading predefined environment variables from files
# Typically used to load sensitive access credentials
%pip install python-dotenv

# Interacting with Trino and using Trino with sqlalchemy
%pip install trino sqlalchemy sqlalchemy-trino

# Pandas and pyarrow
%pip install pandas
%pip install pyarrow

# OS-Climate utilities to make data ingest easier
%pip install osc-ingest-tools

# Python client for Pachyderm
%pip install python_pachyderm

In [2]:
# Report exceptions only in cells output
%xmode Minimal

Exception reporting mode: Minimal


In [3]:
from dotenv import dotenv_values, load_dotenv
import os
import pathlib
import pandas as pd
import trino
from sqlalchemy.engine import create_engine
import osc_ingest_trino as osc
import python_pachyderm

Define Environment and Execution Variables

In [4]:
# Load environment variables from credentials.env
osc.load_credentials_dotenv()

Connect to Pachyderm service using the Python client, retrieve the data from ingested file and load into a dataframe

In [5]:
# Create a pachyderm client by manually setting up host and port
client = python_pachyderm.Client(os.environ['PACH_ENDPOINT'], os.environ['PACH_PORT'])

In [6]:
# Verify the csv file can be read
f = client.get_file(("wri-gppd", "master"), "/global_power_plant_database_v_1_3/global_power_plant_database.csv")
df_gppd = pd.read_csv(f, low_memory=False).convert_dtypes()

In [7]:
print(df_gppd.info(verbose=True))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34936 entries, 0 to 34935
Data columns (total 36 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   country                         34936 non-null  string 
 1   country_long                    34936 non-null  string 
 2   name                            34936 non-null  string 
 3   gppd_idnr                       34936 non-null  string 
 4   capacity_mw                     34936 non-null  Float64
 5   latitude                        34936 non-null  Float64
 6   longitude                       34936 non-null  Float64
 7   primary_fuel                    34936 non-null  string 
 8   other_fuel1                     1944 non-null   string 
 9   other_fuel2                     276 non-null    string 
 10  other_fuel3                     92 non-null     string 
 11  commissioning_year              17447 non-null  Float64
 12  owner                           

In [8]:
df_gppd.head()

,country,country_long,name,gppd_idnr,capacity_mw,latitude,longitude,primary_fuel,other_fuel1,other_fuel2,...,estimated_generation_gwh_2013,estimated_generation_gwh_2014,estimated_generation_gwh_2015,estimated_generation_gwh_2016,estimated_generation_gwh_2017,estimated_generation_note_2013,estimated_generation_note_2014,estimated_generation_note_2015,estimated_generation_note_2016,estimated_generation_note_2017
0,AFG,Afghanistan,Kajaki Hydroelectric Power Plant Afghanistan,GEODB0040538,33.0,32.322,65.119,Hydro,<NA>,<NA>,...,123.77,162.9,97.39,137.76,119.5,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1
1,AFG,Afghanistan,Kandahar DOG,WKS0070144,10.0,31.67,65.795,Solar,<NA>,<NA>,...,18.43,17.48,18.25,17.7,18.29,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE
2,AFG,Afghanistan,Kandahar JOL,WKS0071196,10.0,31.623,65.792,Solar,<NA>,<NA>,...,18.64,17.58,19.1,17.62,18.72,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE
3,AFG,Afghanistan,Mahipar Hydroelectric Power Plant Afghanistan,GEODB0040541,66.0,34.556,69.4787,Hydro,<NA>,<NA>,...,225.06,203.55,146.9,230.18,174.91,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1
4,AFG,Afghanistan,Naghlu Dam Hydroelectric Power Plant Afghanistan,GEODB0040534,100.0,34.641,69.717,Hydro,<NA>,<NA>,...,406.16,357.22,270.99,395.38,350.8,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1


Connect to Trino and load the data

In [9]:
# Define execution variables for loading to Iceberg / Trino
ingest_catalog = 'osc_datacommons_dev'
ingest_schema = 'sandbox'
ingest_table = 'gppd'

In [10]:
engine = osc.attach_trino_engine(verbose=True, catalog=ingest_catalog)

using connect string: trino://caldeirav@trino-secure-odh-trino.apps.odh-cl2.apps.os-climate.org:443/osc_datacommons_dev


In [11]:
# Show available schemas to ensure trino connection is set correctly
schema_show_sql = f"""
show schemas in {ingest_catalog}
"""
schema_show = engine.execute(schema_show_sql)
print(schema_show.fetchall())

[('default',), ('demo_dv',), ('iceberg_demo',), ('information_schema',), ('mdt_sandbox',), ('pcaf_sovereign_footprint',), ('sandbox',)]


Create target table

In [12]:
# Drop table if exists in case we run the pipeline multiple times
sql=f"""
drop table if exists {ingest_catalog}.{ingest_schema}.{ingest_table}
"""
qres = engine.execute(sql)
print(qres.fetchall())

[(True,)]


In [13]:
# Create table definition and create table query
columnschema = osc.create_table_schema_pairs(df_gppd)
tabledef = f"""
create table if not exists {ingest_catalog}.{ingest_schema}.{ingest_table}(
{columnschema}
) with (
    format = 'ORC',
    partitioning = array['country']
)
"""
print(tabledef)


create table if not exists osc_datacommons_dev.sandbox.gppd(
    country varchar,
    country_long varchar,
    name varchar,
    gppd_idnr varchar,
    capacity_mw double,
    latitude double,
    longitude double,
    primary_fuel varchar,
    other_fuel1 varchar,
    other_fuel2 varchar,
    other_fuel3 varchar,
    commissioning_year double,
    owner varchar,
    source varchar,
    url varchar,
    geolocation_source varchar,
    wepp_id varchar,
    year_of_capacity_data bigint,
    generation_gwh_2013 double,
    generation_gwh_2014 double,
    generation_gwh_2015 double,
    generation_gwh_2016 double,
    generation_gwh_2017 double,
    generation_gwh_2018 double,
    generation_gwh_2019 double,
    generation_data_source varchar,
    estimated_generation_gwh_2013 double,
    estimated_generation_gwh_2014 double,
    estimated_generation_gwh_2015 double,
    estimated_generation_gwh_2016 double,
    estimated_generation_gwh_2017 double,
    estimated_generation_note_2013 var

In [14]:
# Execute table creation if the table does not exist
qres = engine.execute(tabledef)
print(qres.fetchall())

[(True,)]


In [15]:
# Filter the dataframe to one country in order to fasten the demo
df_gppd = df_gppd[df_gppd.isin(['France']).any(axis=1)]

In [16]:
# Insert the dataframe data in batches of 100
df_gppd.to_sql(ingest_table,
           con=engine,
           schema=ingest_schema,
           if_exists='append',
           index=False,
           method=osc.TrinoBatchInsert(batch_size = 100, verbose = False))